<a href="https://colab.research.google.com/github/fabiomonte2/AP/blob/main/trabalho_repa_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

import pandas as pd

from pandas import Series, DataFrame

from sklearn.model_selection import train_test_split

# Nova seção

In [2]:
data = pd.read_csv('sales_train.csv')


In [3]:
data.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [4]:
import pandas as pd
data = pd.read_csv('sales_train.csv')
data.dtypes

date               object
date_block_num      int64
shop_id             int64
item_id             int64
item_price        float64
item_cnt_day      float64
dtype: object

In [5]:
y = data['item_price']
y

0           999.00
1           899.00
2           899.00
3          1709.05
4          1099.00
            ...   
2935844     299.00
2935845     299.00
2935846     349.00
2935847     299.00
2935848     299.00
Name: item_price, Length: 2935849, dtype: float64

In [6]:
X = data.drop('item_price',axis=1)
X.columns

Index(['date', 'date_block_num', 'shop_id', 'item_id', 'item_cnt_day'], dtype='object')

In [7]:
for column in X.columns:
  print(f"{column:>12}: {len(set(X[column])):4} {X[column].dtype}")

        date: 1034 object
date_block_num:   34 int64
     shop_id:   60 int64
     item_id: 21807 int64
item_cnt_day:  198 float64


In [8]:
indesejadas = ['item_cnt_day']
Xdrop = X.drop(indesejadas,axis=1)
Xdrop.columns

Index(['date', 'date_block_num', 'shop_id', 'item_id'], dtype='object')

In [9]:
Xnum = Xdrop.select_dtypes('number')
Xnum.columns

Index(['date_block_num', 'shop_id', 'item_id'], dtype='object')

In [10]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median') 
XnumLimpo = imputer.fit_transform(Xnum)
XnumLimpo

array([[    0.,    59., 22154.],
       [    0.,    25.,  2552.],
       [    0.,    25.,  2552.],
       ...,
       [   33.,    25.,  7459.],
       [   33.,    25.,  7440.],
       [   33.,    25.,  7460.]])

In [11]:
for column in Xnum.columns:
  print(f"{column:>12}: {sum(Xnum[column].isnull())}")

date_block_num: 0
     shop_id: 0
     item_id: 0


In [12]:
Xcat = Xdrop.select_dtypes('object')
Xcat.columns

Index(['date'], dtype='object')

In [13]:
for column in Xcat.columns:
  print(f"{column:>12}: {sum(Xcat[column].isnull())}")

        date: 0


In [14]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='most_frequent')
XcatLimpo = imputer.fit_transform(Xcat)
XcatLimpo

array([['02.01.2013'],
       ['03.01.2013'],
       ['05.01.2013'],
       ...,
       ['14.10.2015'],
       ['22.10.2015'],
       ['03.10.2015']], dtype=object)

In [15]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
XcatHot = encoder.fit_transform(XcatLimpo)
XcatHot

<2935849x1034 sparse matrix of type '<class 'numpy.float64'>'
	with 2935849 stored elements in Compressed Sparse Row format>

In [1]:
import numpy as np 
Xtratado = np.c_[XnumLimpo,XcatHot.toarray()]
Xtratado.shape

NameError: ignored

In [ ]:
import pandas as pd
train = pd.read_csv('sales_train.csv')
test = pd.read_csv('test.csv')
train.columns.isin(test.columns)

In [ ]:
# Remover as caracteristicas indesejáveis
from sklearn.base import BaseEstimator, TransformerMixin
class AtributosDesejados(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
    self.colunasIndesejadas = ['item_cnt_day']
    return self
  def transform(self, X, y=None):
    return X.drop(self.colunasIndesejadas,axis=1)

atributosDesejados = AtributosDesejados()
Xdrop = atributosDesejados.fit_transform(X)
Xdrop.columns

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
class AtributosNumericos(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
    self.colunasNumericas = X.select_dtypes(include='number').columns
    return self
  def transform(self, X, y=None):
    return X[self.colunasNumericas]

atributosNumericos = AtributosNumericos()
Xnum = atributosNumericos.fit_transform(Xdrop)
Xnum.columns

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

pipenum = Pipeline([
                ('atributos_numericos', AtributosNumericos()),
                ('imputer', SimpleImputer(strategy='median')),
                ('sacaler', StandardScaler())
])

XnumLimpo = pipenum.fit_transform(Xnum)
XnumLimpo

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
class AtributosCategoricos(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
    self.colunasCategoricas = X.select_dtypes(include='object').columns
    return self
  def transform(self, X, y=None):
    return X[self.colunasCategoricas]

atributosCategoricos = AtributosCategoricos()
Xcat = atributosCategoricos.fit_transform(Xdrop)
Xcat.columns

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

pipecat = Pipeline([
                ('atributos_categoricos', AtributosCategoricos()),
                ('imputer', SimpleImputer(strategy='most_frequent')),
                ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

XcatLimpo = pipenum.fit_transform(Xdrop)
XcatLimpo

In [ ]:
from sklearn.pipeline import FeatureUnion
unecaracteristicas = FeatureUnion([
      ('pipenum', pipenum),
      ('pipecat', pipecat)
])

Xtratado = unecaracteristicas.fit_transform(X)
Xtratado

In [ ]:
from sklearn.pipeline import Pipeline

preproc = Pipeline([
        ('unecaracteristicas', unecaracteristicas)
])

Xtratado = preproc.fit_transform(Xdrop)
Xtratado

In [ ]:
# testando o modelo com o algoritmo de arvore de decisão
from sklearn.tree import DecisionTreeClassifier

pipetotal = Pipeline([
        ('preproc', preproc),
        ('arvore', DecisionTreeClassifier())             
])
from sklearn.model_selection import cross_validate
import numpy as np

scores = cross_validate(pipetotal, X, y)
scores, np.mean(scores['test_score'])

from sklearn.model_selection import GridSearchCV
parametros = {
    'arvore__max_depth': [None] + list(range(1,20,2)),
    'arvore__criterion': ['gini', 'entropy']
    }
modelo = GridSearchCV(pipetotal, param_grid=parametros)
scores = cross_validate(pipetotal, X, y)
scores, np.mean(scores['test_score'])




In [ ]:
from sklearn.neighbors import KNeighborsClassifier

pipetotal = Pipeline([
        ('preproc', preproc),
        ('knn', KNeighborsClassifier())             
])
from sklearn.model_selection import cross_validate
import numpy as np

scores = cross_validate(pipetotal, X, y)
scores, np.mean(scores['test_score'])

from sklearn.model_selection import GridSearchCV
parametros = {
    'knn__max_depth': [None] + list(range(1,20,2)),
    'knn__criterion': ['gini', 'entropy']
    }
modelo = GridSearchCV(pipetotal, param_grid=parametros)
scores = cross_validate(pipetotal, X, y)
scores, np.mean(scores['test_score'])

In [ ]:
from sklearn.linear_model import Perceptron

pipetotal = Pipeline([
        ('preproc', preproc),
        ('perceptron', Perceptron())             
])
from sklearn.model_selection import cross_validate
import numpy as np

scores = cross_validate(pipetotal, X, y)
scores, np.mean(scores['test_score'])

from sklearn.model_selection import GridSearchCV
parametros = {
    'perceptron__max_depth': [None] + list(range(1,20,2)),
    'perceptron__criterion': ['gini', 'entropy']
    }
modelo = GridSearchCV(pipetotal, param_grid=parametros)
scores = cross_validate(pipetotal, X, y)
scores, np.mean(scores['test_score'])

In [ ]:
from sklearn.linear_model import LogisticRegression

pipetotal = Pipeline([
        ('preproc', preproc),
        ('logisticreg', LogisticRegression())             
])
from sklearn.model_selection import cross_validate
import numpy as np

scores = cross_validate(pipetotal, X, y)
scores, np.mean(scores['test_score'])

from sklearn.model_selection import GridSearchCV
parametros = {
    'logisticreg__max_depth': [None] + list(range(1,20,2)),
    'logisticreg__criterion': ['gini', 'entropy']
    }
modelo = GridSearchCV(pipetotal, param_grid=parametros)
scores = cross_validate(pipetotal, X, y)
scores, np.mean(scores['test_score'])

In [ ]:
from sklearn.ensemble import RandomForestClassifier

pipetotal = Pipeline([
        ('preproc', preproc),
        ('randonf', RandomForestClassifier())             
])
from sklearn.model_selection import cross_validate
import numpy as np

scores = cross_validate(pipetotal, X, y)
scores, np.mean(scores['test_score'])

from sklearn.model_selection import GridSearchCV
parametros = {
    'randonf__max_depth': [None] + list(range(1,20,2)),
    'randonf__criterion': ['gini', 'entropy']
    }
modelo = GridSearchCV(pipetotal, param_grid=parametros)
scores = cross_validate(pipetotal, X, y)
scores, np.mean(scores['test_score'])